In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from desilike.theories.galaxy_clustering import FOLPSAXTracerPowerSpectrumMultipoles, DirectPowerSpectrumTemplate, FixedPowerSpectrumTemplate

template = DirectPowerSpectrumTemplate(fiducial = 'DESI', z = 0.5)
theory = FOLPSAXTracerPowerSpectrumMultipoles(template = template,tracer = 'LRG', prior_basis = 'physical')

In [ ]:
template.params['n_s'].update(fixed = False)
template.params['n_s'].update(prior={'dist':'norm','loc':0.9649, 'scale':0.048}) #Planck width x10
theory.params['b3p'].update(fixed = False)

In [ ]:
#Data from Y1 clustering products

import numpy as np
from pypower import PowerSpectrumMultipoles, BaseMatrix
from desilike.observables import ObservableCovariance

data_fn = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/corrected/pkpoles_corrected_LRG_SGC_z0.4-0.6_thetacut0.05.npy'
wmatrix_fn = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/wmatrix_smooth_LRG_SGC_z0.4-0.6_thetacut0.05.npy'
covariance_fn = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/cov_2pt/ezmock/v1/covariance_power_LRG_SGC_z0.4-0.6_default_FKP_lin_thetacut0.05.npy'

In [ ]:
covariance = ObservableCovariance.load(covariance_fn)
covariance = covariance.select(xlim = (0.02, 0.2), projs = [0,2])

In [ ]:
#Defining an observable without window function
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable

observable = TracerPowerSpectrumMultipolesObservable(data=data_fn, 
                                                     covariance=covariance,
                                                     klim={ell: [0.02, 0.2, 0.005] for ell in [0,2]},
                                                     theory=theory,
                                                     wmatrix = wmatrix_fn,
                                                     kin = np.arange(0.001, 0.35, 0.001),
                                                     )

In [ ]:
observable.covariance.shape

In [ ]:
from desilike.emulators import EmulatedCalculator, Emulator, TaylorEmulatorEngine
theory = observable.wmatrix.theory
emulator = Emulator(theory.pt, engine=TaylorEmulatorEngine(method = 'finite', order = 1))
emulator.save('Emulator/FOLPSAX_mf_Taylor_o4_LRG1')
emulator.set_samples()
emulator.fit()

In [ ]:
theory.init.update(pt = emulator.to_calculator())

In [ ]:
theory.params['alpha0p'].update(derived = '.marg')
theory.params['alpha2p'].update(derived = '.marg')
theory.params['alpha4p'].update(derived = '.marg')
theory.params['sn0p'].update(derived = '.marg')
theory.params['sn2p'].update(derived = '.marg')

In [ ]:
# Update namespace of bias parameters (to have one parameter per tracer / z-bin)
for param in theory.init.params:
    # Update latex just to have better labels
    param.update(namespace='{}'.format('LRG1_SGC'),
                 latex=param.latex(namespace=#'\mathrm{{pre}},
                                   '\mathrm{{{}}}, {:d}'.format('LRG', 0), inline=False))

In [ ]:
from desilike.likelihoods import ObservablesGaussianLikelihood
from desilike import setup_logging

setup_logging()

likelihood = ObservablesGaussianLikelihood(observables = [observable])

In [ ]:
template2 = DirectPowerSpectrumTemplate(fiducial = 'DESI', z = 0.5)
theory2 = FOLPSAXTracerPowerSpectrumMultipoles(template = template2, tracer = 'LRG', prior_basis = 'physical')

In [ ]:
template2.params['n_s'].update(fixed = False)
template2.params['n_s'].update(prior={'dist':'norm','loc':0.9649, 'scale':0.048}) #Planck width x10
theory2.params['b3p'].update(fixed = False)

In [ ]:
data_fn_2 = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/corrected/pkpoles_corrected_LRG_NGC_z0.4-0.6_thetacut0.05.npy'


#data_fn_2 = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/corrected/pkpoles_corrected_LRG_NGC_z0.8-1.1_thetacut0.05.npy'
wmatrix_fn_2 = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/baseline_2pt/pk/wmatrix_smooth_LRG_NGC_z0.4-0.6_thetacut0.05.npy'
covariance_fn_2 = '/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5/unblinded/desipipe/cov_2pt/ezmock/v1/covariance_power_LRG_NGC_z0.4-0.6_default_FKP_lin_thetacut0.05.npy'

In [ ]:
covariance_2 = ObservableCovariance.load(covariance_fn_2)
covariance_2 = covariance_2.select(xlim = (0.02, 0.2), projs = [0,2])

In [ ]:
observable2 = TracerPowerSpectrumMultipolesObservable(data=data_fn_2, 
                                                     covariance=covariance_2,
                                                     klim={ell: [0.02, 0.2, 0.005] for ell in [0,2]},
                                                     theory=theory2,
                                                     wmatrix = wmatrix_fn_2,
                                                     kin = np.arange(0.001, 0.35, 0.001),
                                                     )

In [ ]:
observable.data[0].shape

In [ ]:
theory2 = observable2.wmatrix.theory
theory2.init.update(pt = emulator.to_calculator())

In [ ]:
theory2.params['alpha0p'].update(derived = '.marg')
theory2.params['alpha2p'].update(derived = '.marg')
theory2.params['alpha4p'].update(derived = '.marg')
theory2.params['sn0p'].update(derived = '.marg')
theory2.params['sn2p'].update(derived = '.marg')

In [ ]:
# Update namespace of bias parameters (to have one parameter per tracer / z-bin)
for param in theory2.init.params:
    # Update latex just to have better labels
    param.update(namespace='{}'.format('LRG1_NGC'),
                 latex=param.latex(namespace=#'\mathrm{{pre}},
                                   '\mathrm{{{}}}, {:d}'.format('LRG', 0), inline=False))

In [ ]:
likelihood2 = ObservablesGaussianLikelihood(observables = [observable2])

In [ ]:
from desilike.likelihoods import SumLikelihood

Likelihood = SumLikelihood(likelihoods = (likelihood, likelihood2))

Likelihood()

In [ ]:
from desilike.samplers import EmceeSampler

sampler = EmceeSampler(Likelihood,save_fn = 'Chains/LRG1_NGC_SGC_ell02_kmax0.2')
sampler.run(check={'max_eigen_gr': 0.03})